In [ ]:
import numpy as np
import time
import math
import sympy as sp
import random
from itertools import permutations
from multiprocessing import Pool, freeze_support


def final(total):
    def policy(st,mu_a_t,mu_s_t,matrix_p_t):
        at = mu_a_t+ np.dot(matrix_p_t, st-mu_s_t)
        return at

    def reward(mt,bt,ct,at,st):
        rt = mt + np.dot(bt,at)+ np.dot(ct,st)
        return rt

    def f(st,at,mu_s_t1,mu_s_t,mu_a_t,beta_s_t, beta_a_t):
        s_t1 = mu_s_t1 + np.dot(beta_s_t, st-mu_s_t) + np.dot(beta_a_t, at-mu_a_t)
        return s_t1

    def P(perm, i, j):
        #m_index = min(perm.index(i), perm.index(j))
        m_index = perm.index(i)
        if m_index > 0:
            # 切片，选择前面的所有元素
            ele_list = list(perm[0:m_index])
            if j in ele_list:
                ele_list.remove(j)
            return ele_list
        else:
            return list()

    def simulate_begin(mu_s, mu_a, beta_s, beta_a, subset, e_fix):
        for t in range(1, T):
            a[t] = policy(s[t], mu_a[t], mu_s[t], matrix_p[t])
            if t == 1:
                mean = mean_1  # 均值向量
                cov = cov_1
                # tmp表示有哪些要固定住
                tmp = list()
                for i in subset:
                    if i <= num_s:
                        tmp.append(i)
            if t == 2:
                mean = mean_2  # 均值向量
                cov = cov_2
                # tmp表示有哪些要固定住
                tmp = list()
                for i in subset:
                    if num_s < i <= num_s * 2:
                        tmp.append(i)
            if len(tmp) == num_s:
                e = np.zeros(num_s)
                for i in range(len(tmp)):
                    e[i] = e_fix[subset.index(tmp[i])]
            if len(tmp) == 0:
                e = np.random.multivariate_normal(mean, cov, 1)[0]
            if 0 < len(tmp) < num_s:
                if min(tmp) < num_s + 1:
                    given_indices = [x - 1 for x in tmp]
                else:
                    given_indices = [x - num_s - 1 for x in tmp]
                given_values = np.zeros(len(tmp))
                for i in range(len(tmp)):
                    given_values[i] = e_fix[subset.index(tmp[i])]
                c_sample = sample_from_conditional_distribution(mean, cov, given_indices, given_values)
                e = np.zeros(num_s)
                for j in range(num_s):
                    if j in given_indices:
                        e[j] = given_values[given_indices.index(j)]
                    else:
                        e[j] = c_sample[0]
                        c_sample = c_sample[1:]

            if t + 1 < T:
                s[t + 1] = f(s[t], a[t], mu_s[t + 1], mu_s[t], mu_a[t], beta_s[t], beta_a[t]) + e

            r[t] = reward(m_fix[t], b[t], c[t], a[t], s[t])
            R[t] = R[t - 1] + r[t]
        return R[T - 1]

    def g_hat(subset):
        if len(subset)>0:
            e_fix_1 = sample_factor(subset)
            e_fix_2 = sample_factor(subset)
        else:
            e_fix_1 = 0
            e_fix_2 = 0
        y_1 = simulate_begin(mu_s,mu_a,beta_s,beta_a,subset,e_fix_1)
        y_2 = simulate_begin(mu_s,mu_a,beta_s,beta_a,subset,e_fix_1)
        y_3 = simulate_begin(mu_s,mu_a,beta_s,beta_a,subset,e_fix_2)
        return y_1*(y_2-y_3) 

    def sample_factor(subset):
        # 这样抽取的是一个2元的随机样本
        random_samples_1 = np.random.multivariate_normal(mean_1, cov_1, 1)[0]
        random_samples_2 = np.random.multivariate_normal(mean_2, cov_2, 1)[0]
        # 有多少个元素就生成多大的list装固定的factor
        e_fix = np.zeros(len(subset))
        for i in range(len(subset)):
            # 选取每个元素的标号
            factor = subset[i]
            if factor <= num_s:
                e_fix[i] = random_samples_1[factor - 1]
            else:
                e_fix[i] = random_samples_2[factor - num_s - 1]
        # 最后得到的fix的值和subset的排序一一对应
        return e_fix

    def sample_from_conditional_distribution(mean, cov, given_indices, given_values):
        n = len(mean)
        # 确定变量的索引
        unknown_indices = [i for i in range(n) if i not in given_indices]
        a_value = []
        mu1 = []
        mu2 = []
        for i in range(n):
            if i in given_indices:
                mu2.append(mean[i])
                a_value.append(given_values[given_indices.index(i)])
            else:
                mu1.append(mean[i])
        given_indices = sorted(given_indices)
        mu1 = np.array(mu1)
        mu2 = np.array(mu2)
        cov22 = np.zeros((len(given_indices),len(given_indices)))
        cov11 = np.zeros((len(unknown_indices),len(unknown_indices)))
        cov21 = np.zeros((len(given_indices),len(unknown_indices)))
        for i in range(len(given_indices)):
            for j in range(len(given_indices)):
                cov22[i][j] = cov[given_indices[i]][given_indices[j]]
            for j in range(len(unknown_indices)):
                cov21[i][j] = cov[given_indices[i]][unknown_indices[j]]
        for i in range(len(unknown_indices)):
            for j in range(len(unknown_indices)):
                cov11[i][j] = cov[unknown_indices[i]][unknown_indices[j]]
        cov12 = cov21.T
        mu = mu1 + cov12@np.linalg.inv(cov22)@(a_value-mu2)
        sigma = cov11 - cov12@np.linalg.inv(cov22)@cov21
        sample = np.random.multivariate_normal(mu,sigma)
        return sample

    def sample_sd(Sh, Sh2, N):
        over = Sh2 + Sh**2/N
        under = N-1
        return np.sqrt(over/under)

    # 确定m，进行排序，
    # 这个函数可以筛选出我需要几个，具体是哪些factor的shapley value，返回的Sh_choose的数量就是m
    # 这里输入的是shapley的均值
    def percent_choose(Sh_m,percent):
        per = 0
        Sh_choose = []
        i = 1
        while per < percent:
            per = per + np.sort(Sh_m)[-i]/np.sum(Sh_m)
            Sh_choose.append(np.argsort(Sh_m)[-i])
            i = i + 1
        return Sh_choose

    def boundary_index(Sh_m, Sh_choose):
        bound = len(Sh_choose)
        i = np.argsort(Sh_m)[-bound]
        j = np.argsort(Sh_m)[-(bound+1)]
        return i,j

    # N是一个数组,这里暂定一共有6个数
    def budget_allocation(N,incre,std,gap):
        pp = sp.symbols('pp0:6')
        under = (std/gap)**2
        eq_sum = sp.Eq(sum(pp), np.sum(N)+incre)
        eqs = [sp.Eq(pp[i]/under[i], pp[0]/under[0]) for i in range(1, 6)]
        solution = sp.solve([eq_sum] + eqs, pp)
        N_star = np.zeros(6)
        # 这里要考虑四舍五入以及加和的问题
        for i in range(6):
            N_star[i] = round(float(solution[pp[i]]))
        # 多的就给最小的
        while np.sum(N_star) < np.sum(N) + incre:
            tmp = N_star/under
            i = np.argmin(tmp)
            N_star[i] = N_star[i] + 1
        return N_star

    '''
    一些背景信息，不需要进行调整
    '''
    num_s =3
    T = 4
    num_e = T -2
    num_a= 1
    perms = list(permutations(range(1, num_e*num_s+1))) # 1到num_e 的所有排列组合
    num_per = len(perms) # 一共有多少种排列组合的方式


    '''
    影响shapley value的一些参数，可调整
    '''
    #b = np.ones((T,num_a))
    b = 2*np.ones((T, num_a))
    c = np.ones((T,num_s))
    #c[2] = 1*np.array([1,3,5])
    #c[3] = 1*np.array([5,3,1])
    matrix_p = 0.1*np.ones((T, num_a, num_s))
    matrix_p[2] = 0.2*np.array([1,2,3])
    matrix_p[3] = 0.5*np.array([3.5,2,2])
    
    mu_s = np.random.normal(1, 0, size=(T, num_s))
    mu_a = np.random.normal(1, 0, size=(T, num_a))

    beta_s = np.random.normal(0.5, 0, size=(T, num_s, num_s))
    beta_a = np.random.normal(0.5, 0, size=(T, num_s, num_a))

    # 时间1的方差
    mean_1 = np.array([0, 0, 0])  # 均值向量
    c_e1 = np.random.uniform(0.1, 0.1, 3)
    v_e1 = np.random.uniform(0.5, 0.5, 3)
    cov_1 = np.array([[v_e1[0], c_e1[0], c_e1[1]],
                      [c_e1[0], v_e1[1], c_e1[2]],
                      [c_e1[1], c_e1[2], v_e1[2]]])  # 协方差矩阵
    # 时间2的方差
    mean_2 = np.array([0, 0, 0])  # 均值向量
    c_e2 = np.random.uniform(-1, -1, 3)
    v_e2 = np.random.uniform(3, 3, 3)
    cov_2 = np.array([[v_e2[0], c_e2[0], c_e2[1]],
                      [c_e2[0], v_e2[1], c_e2[2]],
                      [c_e2[1], c_e2[2], v_e2[2]]])  # 协方差矩阵
    '''
    一些主观设定
    '''
    N0 = 10000
    delta = 6000
    N = 120000
    correct = 0
    #total = 50
    true_set = [2,1,0,3]
    for _ in range(total):
        R = np.zeros(T)
        r = np.zeros(T)
        s = np.ones((T, num_s))
        a = np.zeros((T, num_a))
        m_fix = np.ones(T)
        Sh_hat = np.zeros(6)
        Sh2 = np.zeros(6)
        N_set = np.zeros(6)
        '''
        一阶段部分
        '''
        stage_1_s = time.time()
        for n in range(N0):
            perm = perms[random.randint(0, num_per - 1)]
            for i in range(1,7):
                N_set[i-1] = N_set[i-1] + 1
                epsilon = g_hat(P(perm,i,0)+[i]) - g_hat(P(perm,i,0))
                Sh_hat[i-1] = Sh_hat[i-1] + epsilon
                Sh2[i-1] = Sh2[i-1] + epsilon**2
        stage_1_e = time.time()
        #print('一阶段结束：',stage_1_e-stage_1_s,'s')

        '''
        二阶段部分
        '''
        stage_2_s = time.time()
        while np.sum(N_set)< N:
            Sh_mean = Sh_hat/N_set
            #print(Sh_mean)
            Sh_std = sample_sd(Sh_hat, Sh2, N_set)
            sigma_hat = Sh_std/np.sqrt(N_set)
            factor_choose = percent_choose(Sh_mean, 0.9) # 解释90%的方差就行
            if len(factor_choose) < 6:
                m, m1 = boundary_index(Sh_mean, factor_choose) # 取出相对应的索引
                boundary = (sigma_hat[m1]*Sh_mean[m] + sigma_hat[m]*Sh_mean[m1])/(sigma_hat[m] + sigma_hat[m1])
            else:
                boundary = np.min(Sh_mean) - 1
            gap = Sh_mean - boundary
            N_star = budget_allocation(N_set, delta, Sh_std, gap)
            incre_round = 0    
            while np.sum(N_set) < np.sum(N_star):
                if incre_round == delta:
                    break
                perm = perms[random.randint(0, num_per - 1)] 
                for i in range(1,7):
                    if incre_round < delta and N_set[i-1] < N_star[i-1]:
                        incre_round += 1
                        N_set[i-1] = N_set[i-1] + 1
                        epsilon = g_hat(P(perm,i,0)+[i]) - g_hat(P(perm,i,0))
                        Sh_hat[i-1] = Sh_hat[i-1] + epsilon
                        Sh2[i-1] = Sh2[i-1] + epsilon**2

        stage_2_e = time.time()
        Sh_mean = Sh_hat/N_set
        factor_choose = percent_choose(Sh_mean, 0.9)


        if set(factor_choose)==set(true_set):
            correct = correct + 1
    return correct

In [ ]:
if __name__ == '__main__':
    s_time = time.time()
    freeze_support()
    num_K = 64
    total = 8
    with Pool(processes= num_K) as pool:
        data = [total for i in range(num_K)]
        result = pool.map(final,data)
    res = 0
    for t in result:
        res += t
    print('准确率为：',res/(num_K*total))
    e_time = time.time()
    print('耗时：',e_time - s_time,'s')